In [10]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import tensorflow_hub as hub

In [11]:
gpu=tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0],True)

In [152]:
model=hub.load('https://tfhub.dev/google/movenet/singlepose/thunder/4')
movenet=model.signatures['serving_default']
# interpreter = tf.lite.Interpreter(model_path="model_float32.tflite")
# interpreter.allocate_tensors()

In [161]:
def draw_keypoints(frame, keypoints, confidence_threshold=0.3):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    for idx in range(shaped.shape[0]):
        if idx==1:continue
        elif idx==2:continue
        elif idx==3:continue
        elif idx==4:continue
        ky, kx, kp_conf = shaped[idx]
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

In [29]:
EDGES = {
    # (0, 1): 'm',
    # (0, 2): 'c',
    # (1, 3): 'm',
    # (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [160]:
def draw_connections(frame, keypoints, edges, confidence_threshold=0.3):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)

In [16]:
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

In [1]:
# cap = cv2.VideoCapture('VIDEO_TEST/dataset/chute01/cam6.avi')
cap = cv2.VideoCapture('VIDEO_TEST/2.mp4')
while cap.isOpened():
    ret, frame = cap.read()
    
    # Resize image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 256,256)
    input_img = tf.cast(img, dtype=tf.int32)
    
    # Detection section
    results = movenet(input_img)
    # keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    keypoints = results['output_0']
    # Render keypoints 
    # loop_through_people(frame, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(frame,keypoints,0.4)
    draw_connections(frame,keypoints,EDGES,0.4)
    
    cv2.imshow('Movenet Multipose', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined